## Goal
Create a new model by **emsenbling 3 models of prediction** with optimized weight.  
We use these models to create a new model. (training of these models are already done)
1. RandomForestClassifier (library: sklearn)
2. XGBoostClassifier (library: dmlc/xgboost)
3. NeuralNetwork (library: keras)

(reference : https://www.kaggle.com/hsperr/otto-group-product-classification-challenge/finding-ensamble-weights )

## Steps
We create a new model by following steps
1. Load 3 models.
2. Find best weight by optimization method
  - We use *Nelder-Mead* method in weight optimization

## 1. Load 3 models

### Prepare helper functions

In [45]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
import os

def print_model_performance(model, train_x, train_y, val_x, val_y):
    print 'Accuracy on training data = {score}'.format(score=accuracy_score(model.predict(train_x), train_y))
    print 'Accuracy on validation data = {score}'.format(score=accuracy_score(model.predict(val_x), val_y))
    print 'LogLoss on training data = {score}'.format(score=log_loss(train_y, model.predict_proba(train_x)))
    print 'LogLoss on validation data = {score}'.format(score=log_loss(val_y, model.predict_proba(val_x)))
    
def save_model(model, name):
    os.system("mkdir -p %s_pickel" % name)
    fpath = os.path.join("%s_pickel" % name, "%s.pkl" % name)
    joblib.dump(model, fpath)

def load_model(name):
    fpath = os.path.join("%s_pickel" % name, "%s.pkl" % name)
    return joblib.load(fpath)

### Data processing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedShuffleSplit
import xgboost as xgb

train  = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
labels = np.array([int(s[-1])-1 for s in train["target"].values])
train.drop(["id", "target"], axis=1, inplace=True)
test.drop(["id"], axis=1, inplace=True)

sss = StratifiedShuffleSplit(labels, test_size=0.05, random_state=1234)
for train_index, validation_index in sss:
    break

train_x, train_y = train.values[train_index], labels[train_index]
val_x, val_y = train.values[validation_index], labels[validation_index]
test_x = test.values
xg_train = xgb.DMatrix(train_x, label=train_y)
xg_val = xgb.DMatrix(val_x, label=val_y)
xg_test = xgb.DMatrix(test_x)

### Load RandomForestClassifier

In [66]:
from sklearn.ensemble import RandomForestClassifier
rfc = GridSearchCV(RandomForestClassifier(), {}, scoring="log_loss")
rfc = load_model(type(rfc.estimator).__name__.lower())
print_model_performance(rfc, train_x, train_y, val_x, val_y)

Accuracy on training data = 0.974840092542
Accuracy on validation data = 0.815772462831
LogLoss on training data = 0.212241486064
LogLoss on validation data = 0.539763757571


### Load XGBoostClassifier

In [65]:
from xgboost import XGBClassifier
bst = GridSearchCV(XGBClassifier(), {}, scoring="log_loss")
bst = load_model(type(bst.estimator).__name__.lower())
print_model_performance(bst, train_x, train_y, val_x, val_y)

Accuracy on training data = 0.909669297768
Accuracy on validation data = 0.825791855204
LogLoss on training data = 0.27363384561
LogLoss on validation data = 0.472169675172


## 2. Find best weight by optimization method

### 2-1 Define objective function to optimize

In [8]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import log_loss

def objective_func(weights, models, X, y):
    weighted_pred = np.array([w*model.predict_proba(X) for w, model in zip(weights, models)]).sum(axis=0)
    return log_loss(y, weighted_pred)

### 2-2 Define function to generate initial value of target variable

In [6]:
import random
def gen_initial_guess(model_num):
    rand_nums = [random.random() for _ in range(model_num)]
    return [num/sum(rand_nums) for num in rand_nums]

### 2-3  Define weight constraints (sum of weights is 1 and weight range is [0,1])
- `{ type = "eq", ... }` means
> Equality constraint means that the constraint function result is to be zero

In [12]:
const = { "type" : "eq", "fun" : lambda weights: 1-sum(weights) }
bounds = [(0,1)] * len(models)

### 2-4 Find best weight by running optimization method

In [72]:
models = [rfc, bst]
minimize_curry = lambda init_state: minimize(objective_func, init_state, method='SLSQP', bounds=bounds, constraints=const, args=(models, val_x, val_y))
results = [minimize_curry(gen_initial_guess(len(models))) for _ in range(10)]
best = sorted(results, key=lambda res: res["fun"])[-1]
print('Best Score: {best_score}'.format(best_score=best['fun']))
print('Best Weights: {best_weights}'.format(best_weights=best['x']))

Best Score: 0.466738680621
Best Weights: [ 0.13204818  0.86795182]


### 2-5 Create a new model

In [57]:
class EmsembledModel:
    
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights
    
    def predict(self, X):
        weighted_prediction = np.array([w*model.predict(X) for w, model in zip(self.weights, self.models)]).sum(axis=0)
        return [round(n) for n in weighted_prediction]
    
    def predict_proba(self, X):
        return np.array([w*model.predict_proba(X) for w, model in zip(self.weights, self.models)]).sum(axis=0)

In [73]:
emsemble_clf = EmsembledModel(models, best['x'])
print_model_performance(emsemble_clf, train_x, train_y, val_x, val_y)

Accuracy on training data = 0.908988840501
Accuracy on validation data = 0.821913380737
LogLoss on training data = 0.257721155409
LogLoss on validation data = 0.466738680621


## Submit answer

In [74]:
def write_out_prediction(predict_probability, filename="ans.csv"):
    cols = ["id"] + ["Class_%d"%i for i in range(1,10)]
    vals = np.c_[np.arange(start=1, stop=predict_probability.shape[0]+1), predict_probability]
    ans = pd.DataFrame(vals, columns=cols, dtype=float)
    ans["id"] = ans["id"].astype(int)
    ans.to_csv(filename, index=False)

In [75]:
write_out_prediction(emsemble_clf.predict_proba(test_x), "emsemble.csv")

`>>> score is 0.46091`